In [2]:
import pandas as pd
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import nltk



In [91]:
df = pd.read_csv("../../datasets_2/Food Recipes/5000 Indian Cuisines Dataset(with images)/cuisine_updated.csv")

In [37]:
'''
# Create a dictionary for the words you want to replace
replacement_dict = {
    "Non Vegeterian": "Non-Vegetarian",
    "No Onion": "No-Onion",
    "No Garlic": "No-Garlic",
    "High Protein": "High-Protein"
}

# Replace the words in the 'diet' column
df['diet'] = df['diet'].replace(replacement_dict, regex=True)
'''

'\n# Create a dictionary for the words you want to replace\nreplacement_dict = {\n    "Non Vegeterian": "Non-Vegetarian",\n    "No Onion": "No-Onion",\n    "No Garlic": "No-Garlic",\n    "High Protein": "High-Protein"\n}\n\n# Replace the words in the \'diet\' column\ndf[\'diet\'] = df[\'diet\'].replace(replacement_dict, regex=True)\n'

In [38]:
df.head(1)

,name,image_url,description,cuisine,course,diet,prep_time,ingredients,instructions,image_available
0,Doddapatre Tambuli Recipe (Karpooravalli Thayi...,https://www.archanaskitchen.com/images/archana...,Doddapatre Tambuli Recipe is a popular healing...,Indian,Side Dish,Vegetarian,Total in 30 M,\n\n\t\t\t\t\t\t\t \t\t4 cups Indian borage (D...,"To begin making the Doddapatre Tambuli, first ...",1


In [39]:
df.shape

(4466, 10)

In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4466 entries, 0 to 4465
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   name             4466 non-null   object
 1   image_url        4466 non-null   object
 2   description      4466 non-null   object
 3   cuisine          4458 non-null   object
 4   course           4424 non-null   object
 5   diet             4424 non-null   object
 6   prep_time        4454 non-null   object
 7   ingredients      4466 non-null   object
 8   instructions     4466 non-null   object
 9   image_available  4466 non-null   int64 
 10  clean_prep_time  4466 non-null   int64 
dtypes: int64(2), object(9)
memory usage: 383.9+ KB


In [57]:
temp = df.iloc[1039]
for i in temp:
    print("#/n",i)

#/n Chai Ka Masala Recipe - Homemade Spice Powder for Tea
#/n https://www.archanaskitchen.com/images/archanaskitchen/BasicRecipes_Spice_Powders/Tea_Masala_Powder_Recipe_Archanas_Kitchen.jpg
#/n On cold winter days and in wet monsoons, a hot steaming cup of Masala tea is what you need to give that instant energy and warmth. Addition of spices make the tea healthier.The secret behind a perfect cup of Masala tea is the Chai ka Masala or Masala Tea Powder.You can store it for 4-6 months in an air-tight container.
#/n nan
#/n nan
#/n nan
#/n nan
#/n 1/4 cup cardamom (elaichi) pods/seeds  1/4 cup cloves (laung)  1/4 cup dry ginger powder  1 tablespoon whole black peppercorns  2 inch cinnamon stick (dalchini)  1 teaspoon nutmeg powder
#/n To begin making Chai ka Masala, place a pan on medium heat. Add cloves, cardamom, black peppercorns and cinnamon, one by one, in no particular order.Dry roast the spices on medium heat for a couple of minutes until they release a roasted aroma. Turn off the 

# Clean and Format the Data

In [42]:
# Handle missing values, especially in columns like ingredients or prep_time.

In [43]:
df['ingredients'] = df['ingredients'].str.replace(r'\n|\t', '', regex=True).str.strip().str.lower()


In [51]:
df.head(2)

,name,image_url,description,cuisine,course,diet,prep_time,ingredients,instructions,image_available
0,Doddapatre Tambuli Recipe (Karpooravalli Thayi...,https://www.archanaskitchen.com/images/archana...,Doddapatre Tambuli Recipe is a popular healing...,Indian,Side Dish,Vegetarian,Total in 30 M,"4 cups indian borage (doddapatre) , cleaned an...","To begin making the Doddapatre Tambuli, first ...",1
1,Fish Tandoori Recipe,https://www.archanaskitchen.com/images/archana...,Have you ever tried out fish tandoori? Trust m...,North Indian Recipes,Appetizer,High Protein Non Vegetarian,Total in 80 M,"1 fish , nicely washed with bones on (i used p...","To begin making the Fish Tandoori recipe, keep...",1


In [117]:
list_prep_time = df['prep_time'].astype(str).values.tolist()  # Convert all values to string
new_list_prep_time = []
for time_str in list_prep_time:
    try:
        new_list_prep_time.append(int(time_str.split(' ')[-2]))  # Extract the numeric part before 'min'
    except (ValueError, IndexError):
        new_list_prep_time.append(0)  # Handle errors in case the format is unexpected


In [119]:
df['clean_prep_time'] = new_list_prep_time

In [120]:
df['clean_prep_time']

0        30
1        80
2        55
3        20
4        35
       ... 
4461     40
4462    530
4463     40
4464     60
4465     40
Name: clean_prep_time, Length: 4466, dtype: int64

In [121]:
df.head(2)

,name,image_url,description,cuisine,course,diet,prep_time,ingredients,instructions,image_available,clean_prep_time
0,Doddapatre Tambuli Recipe (Karpooravalli Thayi...,https://www.archanaskitchen.com/images/archana...,Doddapatre Tambuli Recipe is a popular healing...,Indian,Side Dish,Vegetarian,Total in 30 M,\n\n\t\t\t\t\t\t\t \t\t4 cups Indian borage (D...,"To begin making the Doddapatre Tambuli, first ...",1,30
1,Fish Tandoori Recipe,https://www.archanaskitchen.com/images/archana...,Have you ever tried out fish tandoori? Trust m...,North Indian Recipes,Appetizer,High Protein Non Vegetarian,Total in 80 M,\n\n\t\t\t\t\t\t\t \t\t1 Fish\t\t\t\t\t\t\t \t...,"To begin making the Fish Tandoori recipe, keep...",1,80
